# Temporal models 
Several temporal with the derived features as input, predicting a binary sleep deprivation label

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, SimpleRNN

In [10]:
import gc
gc.collect()
tf.keras.backend.clear_session()
from numba import cuda 
device = cuda.get_current_device()
device.reset()

## Helper functions

In [11]:
def load_embeddings(subject):
    path = '../embeddings/embeddings_' + subject
    normal_embs = np.load(path + '_normal.npy')
    sleepy_embs = np.load(path + '_sleepy.npy')

    return normal_embs, sleepy_embs

def load_multi_embeddings(subjects):
    normal_dict = {}
    sleepy_dict = {}

    for sub in subjects:
        path = '../embeddings/embeddings_sub' + str(sub)
        normal_frames = np.load(path + '_normal.npy')
        sleepy_frames = np.load(path + '_sleepy.npy')

        normal_dict[str(sub)] = normal_frames
        sleepy_dict[str(sub)] = sleepy_frames

    return normal_dict, sleepy_dict

## First model: GRU deep features

### Model initiation

In [12]:
import gc
gc.collect()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, GRU
from keras.callbacks import EarlyStopping

class DrowsinessDetector:
    def __init__(self, segment_length):
        self.segment_length = segment_length
        self.input_shape = (self.segment_length, 2048)
        
    def split_frames(self, frames):
        num_segments = len(frames) // self.segment_length
        frames = frames[:num_segments * self.segment_length]
        return np.array(np.split(frames, num_segments))
    
    def create_labels(self, num_segments, label):
        return np.array([label] * num_segments)
    
    def shuffle_data(self, X, y):
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        return X[indices], y[indices]

    def construct_model(self):
        model = Sequential()
        model.add(SimpleRNN(32, input_shape=self.input_shape))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

    def get_data_one_sub(self, normal_frames, sleepy_frames):
        X_normal = self.split_frames(normal_frames)
        y_normal = self.create_labels(len(X_normal), 0)
        X_sleepy = self.split_frames(sleepy_frames)
        y_sleepy = self.create_labels(len(X_sleepy), 1)
        X = np.concatenate((X_normal, X_sleepy))
        y = np.concatenate((y_normal, y_sleepy))
        X, y = self.shuffle_data(X, y)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        return X_train, X_test, y_train, y_test

    def get_data_multi_sub(self, normal_frames_dict, sleepy_frames_dict, leave_out_ratio=0.3):
            subjects = list(normal_frames_dict.keys())
            leave_out_subjects = np.random.choice(subjects, int(len(subjects) * leave_out_ratio), replace=False)
            print("Test subjects", leave_out_subjects)

            X_train, y_train = None, None
            for subject in subjects:
                if subject in leave_out_subjects:
                    continue
                print('train', subject)
                frames = normal_frames_dict[subject]
                X = self.split_frames(frames)
                y = self.create_labels(len(X), 0)
                if X_train is None:
                    X_train, y_train = X, y
                else:
                    X_train = np.concatenate((X_train, X))
                    y_train = np.concatenate((y_train, y))

                frames = sleepy_frames_dict[subject]
                X = self.split_frames(frames)
                y = self.create_labels(len(X), 1)
                X_train = np.concatenate((X_train, X))
                y_train = np.concatenate((y_train, y))

            X_test, y_test = None, None
            for subject in leave_out_subjects:
                print('test', subject)
                frames = normal_frames_dict[subject]
                X = self.split_frames(frames)
                y = self.create_labels(len(X), 0)

                if X_test is None:
                    X_test, y_test = X, y
                else:
                    X_test = np.concatenate((X_test, X))
                    y_test = np.concatenate((y_test, y))
                    
                frames = sleepy_frames_dict[subject]
                X = self.split_frames(frames)
                y = self.create_labels(len(X), 1)
                
                X_test = np.concatenate((X_test, X))
                y_test = np.concatenate((y_test, y))

            X_train, y_train = self.shuffle_data(X_train, y_train)
            X_test, y_test = self.shuffle_data(X_test, y_test)

            return X_train, X_test, y_train, y_test
    
    def train(self, X_train, y_train, num_epochs=10, batch_size=32, validation_split=0.2):
        self.batch_size = batch_size
        model = self.construct_model()
        early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

        model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_split=validation_split, callbacks=[early_stop])
        
        self.model = model
        print("Done with training")
        return model
    
    def test(self, X_test, y_test):
        model = self.model
        return model.evaluate(X_test, y_test, batch_size=self.batch_size)
    
    def get_f1(self, X_test, y_test):
        y_predict = model.predict(X_test, batch_size = self.batch_size)
        y_predict_int = np.round(y_predict).astype(int).flatten()
        f1 = f1_score(y_test, y_predict_int)
        return f1

In [ ]:
detector = DrowsinessDetector(segment_length=60 * 46)
#normal_embs, sleepy_embs = load_embeddings('sub10')
#X_train, X_test, y_train, y_test = detector.get_data_one_sub(normal_embs, sleepy_embs)

subjects = [9,10, 11, 12, 14,15,16,20,23,24, 25]

#subjects = [1,3,4,6]
normal_dict, sleepy_dict = load_multi_embeddings(subjects)
X_train, X_test, y_train, y_test = detector.get_data_multi_sub(normal_dict, sleepy_dict)

model = detector.train(X_train, y_train, num_epochs=100, batch_size=8)
#detector.test(X_test, y_test)
#y_test, y_predict = detector.compare_predictions(X_test, y_test)
#print(y_test, y_predict)

Test subjects ['25' '14' '10']
train 9
train 11
train 12
train 15
train 16
train 20
train 23
train 24
test 25
test 14
test 10


In [8]:
# import gc
# gc.collect()
# y_predict = model.predict(X_test)
# y_predict_int = np.round(y_predict).astype(int).flatten()
# f1 = f1_score(y_test, y_predict_int)

In [9]:
import gc
gc.collect()
f1 = detector.get_f1(X_test, y_test)
print(f1)

KeyboardInterrupt: 

In [7]:
print(f1)

0.5040650406504066
